# Example of processing user interface changes into Pandas changes

This notebook explores how a JSON file returned from the User Interface, representing its state could be used to make changes to the panda dataframes that are used as the primary input into the optimizer model.

At the time of writing it was clear that the UI would be unable to generate test data to build this capability within the three weeks of the project, so we have assumed that the output files from the UI will look similar to those that were passed to it i.e. they will be in a hierarchical JSON format.

For simplicity, the code below focuses on a single, but reasonably complex, state change to give the reader an idea as to how the pattern could be extended to the rest of the tool.  Clearly only state that can be changed in the end user interface needs to be handled by programs like this one.  It should be possible, therefore, to extend this mechanism as the user interface becomes more comprehensive.

In [1]:
import workforce_pandas as wfpd
import numpy as np
import math
wfpd.sheets

['pop_chronic_trend',
 'pop_chronic_prev',
 'chron_care_freq',
 'geo_area_list',
 'service_characteristics',
 'pop_acute_need',
 'frontsheet',
 'population',
 'provider_supply',
 'pop_prev_need',
 'provider_list',
 'encounter_detail',
 'metadata',
 'overhead_work',
 'encounter_types',
 'index']

As the return data is quite large, it is more convenient to play with it as a file.  It is recommended that a JSON response is captured from the main data requests APIs (provider_profile, geo_profile) and then modified in an online JSON editor to add, change and remove information.  This will provide a good test file.

In this example we are going to look at provider/type to service mapping and understand what changes (if any) have been made in the user interface.

The instruction below loads the JSON file as a Pyton data dictionary for manipulation.

In [2]:
import json

with open('../data/my_output2.json') as json_data:
    my_dict = json.load(json_data)

The format of the resulting dictionary is shown in the cell below:

In [3]:
my_dict

{'request': {'request_type': 'provider_profile'},
 'response': {'Phys': {'provider_type': 'Physician',
   'services:': [{'Counseling': [{'Diet/Physical Activity Counseling': {'min_f2f_time': 5,
        'max_f2f_time': 5,
        'score': '0.50'}},
      {'Advanced Care Planning': {'min_f2f_time': 5,
        'max_f2f_time': 5,
        'score': '0.50'}},
      {'Anticipatory Guidance': {'min_f2f_time': 5,
        'max_f2f_time': 5,
        'score': '0.50'}},
      {'BRCA risk assessment and genetic counseling': {'min_f2f_time': 5,
        'max_f2f_time': 5,
        'score': '0.50'}},
      {'Breast Cancer Preventive Medications': {'min_f2f_time': 5,
        'max_f2f_time': 5,
        'score': '0.50'}},
      {'Breast Feeding Interventions': {'min_f2f_time': 5,
        'max_f2f_time': 5,
        'score': '0.50'}},
      {'Condition Consultation': {'min_f2f_time': 10,
        'max_f2f_time': 15,
        'score': '0.50'}},
      {'Folic Acid Supplementation': {'min_f2f_time': 5,
        'ma

To navigate the JSON structure, it will be useful to have a list of the provider types abbreviations; these can be extracted quickly from the provider list dataframe.

In [4]:
provider_list = wfpd.dataframes["provider_list"]["provider_abbr"]
list(provider_list)

['Phys',
 'PA',
 'NP',
 'RN',
 'PharmD',
 'MA',
 'Educ',
 'Psych',
 'LCSW',
 'CMHC',
 'MFT']

The data we are going to compare against is stored in the service characteristics dataframe, so we give it an easy to rememebr variable name (and no, I can't remember why I called it total wage - probably an early morning cut and paste without thinking!)

In [5]:
total_wage = wfpd.dataframes["service_characteristics"]
total_wage

,svc_category,svc_desc,min_f2f_time,max_f2f_time,Phys,PA,NP,RN,PharmD,MA,Educ,Psych,LCSW,CMHC,MFT
0,Behavioral Health Screenings,Alcohol Misuse Screening,2.0,2.0,0.5,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
1,Behavioral Health Screenings,Autism Spectrum Disorder Screening,5.0,5.0,0.5,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
2,Behavioral Health Screenings,Depression Screening,4.0,4.0,0.5,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
3,Behavioral Health Screenings,Child Developmental Screening,5.0,5.0,0.5,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
4,Behavioral Health Screenings,Child Developmental Surveillance,2.0,2.0,0.5,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
5,Behavioral Health Screenings,Intimate partner violence screening,2.0,2.0,0.5,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
6,Behavioral Health Screenings,Maternal Depression Screening,2.0,2.0,0.5,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
7,Behavioral Health Screenings,Post Partum Depression Screening,5.0,5.0,0.5,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN
8,Behavioral Health Screenings,Psychosocial/Behavioral Surveillance,2.0,2.0,0.5,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
9,Behavioral Health Screenings,Tobacco/Alcohol/Drug Use,3.0,3.0,0.5,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5


The routine below cycles through the the existing service characteristics dataframe one row at a time.  For each column that corresponds to a profession type, it reads the existing value for suitability.

As the JSON only contains the information with valid entries and a number of arrays (i.e. NaNs are not stored), the parsing is not necessarily straightforward.  The nested loops below basically parse the JSON tree using the values from the base dataframe, detecting dead ends in the JSON data structure and then moving on.

This routine successfully identifies:
* When suitability for a provider_type has been removed (i.e there is no entry in the JSON, but there is in the dataframe)
* When suitability for a provider_type has been changed (i.e. when the entries in the JSON and dataframe don't match)
* When suitability for a provider_type has been created (i.e. when the JSON contains an entry, but the dataframe contains NaN)

It will _not_ automatically identify completely new services (i.e. rows) that were not present in the imported dataframes.  It is suggested that a separate routine should identify new services created in the UI; this could run _before_ this routine and would could update the services rows in the dataframes.  This would then enable this service to pick up the new relationships.  A similar routine could be used for providers too...

Clearly this routine, as written, only considers the suitability figures.  The same pattern should be able to be deployed for the other columns in the sheet (and indeed for the rest of the UI scope).  As they say in the best text books - this is left as an exercise for the reader.

In [6]:
not_defined_in_json = "not defined in json"

In [7]:
for row_index, row in total_wage.iterrows():
    #print (row['min_f2f_time'], row['max_f2f_time'])
    #print ("==========================================================")
    #print (str(row['svc_category'] +"/"+ str(row['svc_desc'])))
    for column in total_wage[provider_list]:
        #score = my_dict.get('response').get(column).get('services:')[0].get(row['svc_category'])[0].get(row['svc_desc'].get('score'))
        new_value = not_defined_in_json
        old_value = row[column]
        #print (type(old_value),old_value)
        #print (column + " old value : " + str(old_value))
        category_size = int(len(my_dict.get('response').get(column).get('services:')))
        #print ("Category: " + str(category_size))
        for n in range(category_size):
            in_json1 = my_dict.get('response').get(column).get('services:')[n].get(row['svc_category'])
            if in_json1 != None :
                description_size = int(len(my_dict.get('response').get(column).get('services:')[n].get(row['svc_category'])))
                #print ("Svc desc: " + str(description_size
                for m in range(description_size):
                    in_json2 = my_dict.get('response').get(column).get('services:')[n].get(row['svc_category'])[m]
                    if in_json2 != None :
                        #print(in_json2)
                        in_json3 = my_dict.get('response').get(column).get('services:')[n].get(row['svc_category'])[m].get(row['svc_desc'])
                        if in_json3 != None :
                            new_value = my_dict.get('response').get(column).get('services:')[n].get(row['svc_category'])[m].get(row['svc_desc']).get('score')
                            string = "Comparison: " + column + " : " + str(old_value) + " -> " + str(new_value)
        if (new_value != old_value):
            if not ((math.isnan(old_value)) and (new_value == not_defined_in_json)):
                if new_value == "not defined in json":
                    total_wage.loc[row_index,column] = np.NaN
                else:
                    total_wage.loc[row_index,column] = new_value 
        

The total_wage is a variable representation of the dataframe, so updating the total_wage dataframe has updated the wfpd.dataframe that is used as a parameter for the optimization calculator.

In [8]:
wfpd.dataframes["service_characteristics"]

,svc_category,svc_desc,min_f2f_time,max_f2f_time,Phys,PA,NP,RN,PharmD,MA,Educ,Psych,LCSW,CMHC,MFT
0,Behavioral Health Screenings,Alcohol Misuse Screening,2.0,2.0,NaN,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
1,Behavioral Health Screenings,Autism Spectrum Disorder Screening,5.0,5.0,NaN,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
2,Behavioral Health Screenings,Depression Screening,4.0,4.0,NaN,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
3,Behavioral Health Screenings,Child Developmental Screening,5.0,5.0,NaN,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
4,Behavioral Health Screenings,Child Developmental Surveillance,2.0,2.0,NaN,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
5,Behavioral Health Screenings,Intimate partner violence screening,2.0,2.0,NaN,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
6,Behavioral Health Screenings,Maternal Depression Screening,2.0,2.0,NaN,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
7,Behavioral Health Screenings,Post Partum Depression Screening,5.0,5.0,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN
8,Behavioral Health Screenings,Psychosocial/Behavioral Surveillance,2.0,2.0,NaN,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
9,Behavioral Health Screenings,Tobacco/Alcohol/Drug Use,3.0,3.0,NaN,0.5,0.5,0.5,NaN,NaN,NaN,0.5,0.5,0.5,0.5
